# A Julia birthday

Execute this notebook by repeatedly pressing shift + enter.

First, load some packages (this might take a while if this is your first Julia notebook on Juliabox). 

In [ ]:
using PyPlot
using Interact

Next, read in the mesh.

In [ ]:
p = readdlm("mesh.node");
t = readdlm("mesh.elem",Int64);
b = readdlm("boundary.elem",Int64); b = unique(b);

Function to compute element stiffness matrices.

In [ ]:
function stiffness_matrix(vertices::Matrix{T}) where {T} 
    d = size(vertices,2)
    M = vcat(ones(T,1,d+1),vertices')
    G = M \ vcat(zeros(T,1,d),eye(T,d))

    return det(M) * Symmetric(G*G') / prod(1:d)
end

Function to compose the finite element matrix and right hand side. 

In [ ]:
function compose_fem_matrix(nodes::Matrix{T}, elements::Matrix{N}, f::Function, boundary::Vector{N}) where {T<:AbstractFloat,N<:Integer}
    
    nnodes,d = size(nodes)
    b = zeros(nnodes)
    I = Vector{Float64}()
    J = Vector{Float64}()
    V = Vector{Float64}()
    
    # loop over elements and compute element stiffness matrices
    for e in 1:size(elements,1)
        element = elements[e,:]
        coords = nodes[element,:]
        S = stiffness_matrix(coords)
        for i in 1:d+1
            for j in 1:d+1
                push!(I,element[i])
                push!(J,element[j])
                push!(V,S[i,j])
            end
        end
        M = vcat(ones(T,1,d+1),coords')
        b[element] += det(M)*f(sum(coords,2)/4)/24
    end
    
    # compose fem matrix
    A = sparse(I,J,V)
    
    # apply Dirichlet condition 
    free_nodes = setdiff(1:nnodes,boundary)
    A2 = speye(A)
    A2[free_nodes,free_nodes] = A[free_nodes,free_nodes]
    
    return A2, b
end

Compose the FEM matrix...

In [ ]:
A, v = compose_fem_matrix(p, t, x -> ones(size(x,1)), b)

... and find a solution.

In [ ]:
f = A\v
f[b] = 0. # force boundary conditions towards 0

Finally, use the `Interact` package to make an interactive plot of the solution.

Use the sliders for the azimuth $\theta$ and elevation $\phi$ to change the view. It might take a while to update the plot...

In [ ]:
fig = figure()
@manipulate for θ = slider(0:20:360,value=260,label="θ"), ϕ = slider(0:10:90,value=50,label="ϕ")
    withfig(fig) do
        ax = subplot(111, projection="3d")
        ax[:azim] = θ; ax[:elev] = ϕ
        plot_trisurf(p[:,1],p[:,2],f,triangles=t-1,cmap=get_cmap("viridis"),edgecolors="k",linewidth=0.1)
    end
end